# SpikeInterface Processing Pipeline for Tetrodes & Neuronexus Probe
### Jake Swann, 2023

##### This is a notebook which takes a spreadsheet as input with information on raw Axona recordings, and sorts all unsorted recordings in a loop. It will concatenate all recordings made on each unique day, and sort them all together, to be split apart afterwards
##### Each path in the spreadsheet should be to a folder containing raw Axona recordings (1 .set and 1 .bin file per recording)
##### The script loads them as a [SpikeInterface](https://github.com/SpikeInterface) object & attaches probe geometry, spike sorts using [Klusta](https://github.com/kwikteam/klusta/), and allows curation of the output in the [phy](https://github.com/cortex-lab/phy/) kwik-gui
##### **N.B.** This requires a Python 3.7 environment with various dependencies installed, including Klusta version 0.95.0, SpikeInterface, and phy. There is code to install most of these from within the 'axona_spikeinterface.ipynb' notebook
---

#### Run axona_spikeinterface script on all recordings marked on a spreadsheet

In [ ]:
import pandas as pd
import numpy as np
import spikeinterface as si
from spikeinterface.extractors import read_axona
from axona_preprocessing import *
from pos_from_bin import *
import os


# sheet = pd.read_excel('tetrode_data_JS.xlsx')
# sheet = pd.read_excel('probe_data_IV.xlsx')
sheet = gs_to_df('https://docs.google.com/spreadsheets/d/1_Xs5i-rHNTywV-WuQ8-TZliSjTxQCCqGWOD2AL_LIq0/edit#gid=0')
path_to_data = '/home/isabella/Documents/isabella/jake/recording_data/'
sorting_suffix = 'sorting_ks2_custom'

sheet['path'] = path_to_data + sheet['path']

sheet_inc = sheet[sheet['Sort'] == 'Y']
trial_list = sheet_inc['trial_name'].to_list()
session_list = np.unique([f"{i.split('_')[0]}_{i.split('_')[1]}" for i in trial_list])    

# Collect into sessions and preprocess
recording_list = []
for i,session in enumerate(session_list):
    recording_list.append([])
    for j in trial_list:
        if session in j:
            base_folder = sheet_inc[sheet_inc['trial_name'] == j]['path'].tolist()[0]
            num_channels = int(sheet_inc[sheet_inc['trial_name'] == j]['num_channels'].tolist()[0])
            electrode_type = sheet_inc[sheet_inc['trial_name'] == j]['probe_type'].tolist()[0]
            print(f"{base_folder}/{j}.set")
            recording = read_axona(f"{base_folder}/{j}.set")
            
            #Generate .pos file if not already present
            if os.path.isfile(f'{base_folder}/{j}.pos') == 0:
                pos_from_bin(f'{base_folder}/{j}')
            
            #Invert trace if recorded on -signal or ref-signal mode (mode 2 and 3)
            # if int(get_mode(f"{base_folder}/{j}.set")) in [2,3]:
#             recording = spre.scale(recording, gain=-1)
#             print('Inverting recording trace')
            
            
            recording_list[i].append([preprocess(recording = recording,
                                     recording_name = j,
                                     base_folder = base_folder,
                                     electrode_type = electrode_type,
                                     num_channels = num_channels),
                                     j, base_folder, electrode_type])

In [ ]:
#Restart kernel so ipython can find the newly written files
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# Concatenate over a single session and sort
for i, recording in enumerate(recording_list):
    recordings_concat = []
    session = pd.DataFrame(recording)
    recordings_concat.append(si.concatenate_recordings(session.iloc[:,0]))
    for j, concat in enumerate(recordings_concat):
        print(f'Sorting {concat}')
        sorting = sort(recording = concat, 
             recording_name = session.iloc[0,1], 
             base_folder = session.iloc[0,2],
             electrode_type = session.iloc[0,3],
             sorting_suffix = sorting_suffix)
    session.to_csv(f'{session.iloc[0,2]}/{session.iloc[0,1][:6]}_{sorting_suffix}/session.csv') #save session trial info to .csv


##### Misc utils

In [ ]:
## Run phy on klusta output
# from phy.apps.kwik import kwik_gui
# os.environ["QTWEBENGINE_CHROMIUM_FLAGS"] = "--single-process"

# ## If tetrodes sorted separately, change channel_group to choose tetrode to display 
# kwik_gui(f'{base_folder}/{session_list[0]}_{sorting_suffix}/recording.kwik', channel_group=15, clustering=None)

# from phy.apps.template import template_gui
# template_gui(f'{base_folder}/{session_list[0]}_{sorting_suffix}/params.py')

import spikeinterface.sorters as ss
# ss.available_sorters()
#ss.installed_sorters()
ss.get_default_sorter_params('kilosort3')
# ss.get_sorter_params_description('kilosort2')
rmdir directory_name

#### Run phy on sorted tetrode recordings

In [ ]:
import shutil

channel_group = 7 #Leave as 0 or set to None for probe recordings
folder = '/home/isabella/Documents/isabella/jake/recording_data/r1299/2023-01-24/230124_sorting_klusta_1.5_3.5'

from phy.apps.kwik import kwik_gui
kwik_gui(f'{folder}/recording.kwik',
                       channel_group=channel_group, clustering=None)
shutil.copy(f'{folder}/cluster_info.tsv', f'{folder}/cluster_info_{channel_group}.tsv')

#### Concatenate cluster info files for tetrode recordings (once curated)

In [ ]:
from utils import concat_cluster

# folder = ''
concat_cluster(folder)

In [ ]:
session.to_csv(f'{session.iloc[0,2]}/{session.iloc[0,1][:6]}_{sorting_suffix}/session.csv')